Packages, Read Data, Arrays

In [1]:
import csv
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
results = []
with open("D:/christodfiles/Desktop/Σχολή/4οΈτος/ΜηχανικήΜάθηση/Εργασία1//abaloneData.csv", "r") as myfile:
    data = csv.reader(myfile, delimiter=",")
    for dataline in data:
        results.append(dataline)
resultsArray = np.array(results)
resultsArrayDel = np.delete(resultsArray, 0, 1)

Max Target, Min Target, number of variables and range of Υ

In [2]:
cells_X = resultsArrayDel[:,:7].astype(float) #deleting 'sex' from data
cells_y = resultsArrayDel[:, 7:].astype(float)
maxY = np.max(cells_y)
minY = np.min(cells_y)
rangeY = maxY - minY
num_columnsX = np.shape(cells_X)[1]
num_columnsY = np.shape(cells_y)[1]
print("Num of Variables: " +str(num_columnsX + num_columnsY))
print("Num of Elements: " +str(len(cells_y)))
print("Max Rings: " +str(maxY))
print("Min Rings: " +str(minY))
print("Range of Y: " +str(rangeY))

Num of Variables: 8
Num of Elements: 4177
Max Rings: 29.0
Min Rings: 1.0
Range of Y: 28.0


Mean, STD for X and Y

In [3]:
meanX = np.mean(cells_X, axis=1)
stdX = np.std(cells_X, axis=1)
meanY = np.mean(cells_y)
stdY = np.std(cells_y)
print("Mean X: " +str(meanX))
print("Std X: " +str(stdX))
print("Mean Y: " +str(meanY))
print("Std Y: " +str(stdY))

Mean X: [0.27207143 0.16407143 0.33857143 ... 0.511      0.49178571 0.7465    ]
Std X: [0.15964792 0.1071659  0.19322608 ... 0.30153169 0.29056741 0.53808045]
Mean Y: 9.933684462532918
Std Y: 3.2237830658212117


Split to training/test set and Normalization (mean=0 and std=1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# Split the data into training/testing sets at 10%
n_samples = np.shape(cells_y)[0]
test_set_size = int(np.round(n_samples*0.2))

cells_X_train = cells_X[:-test_set_size]
cells_X_test = cells_X[-test_set_size:]

# Split the targets into training/testing sets
cells_y_train = cells_y[:-test_set_size]
cells_y_test = cells_y[-test_set_size:]

#normalization
datascaler = StandardScaler().fit(cells_X_train)
cells_X_train_scaled = datascaler.transform(cells_X_train)
cells_X_test_scaled = datascaler.transform(cells_X_test)

print("Training data shape:", np.shape(cells_X_train_scaled))
print("Test data shape:", np.shape(cells_X_test_scaled))

print(cells_X_train_scaled.mean(axis=0)) #mean after transformation
print(cells_X_train_scaled.std(axis=0)) #std after transformation

Training data shape: (3342, 7)
Test data shape: (835, 7)
[-2.71213997e-15 -5.11178472e-15  1.28881213e-14 -4.51714934e-15
  3.22645694e-15 -7.91058820e-16  1.16370774e-15]
[1. 1. 1. 1. 1. 1. 1.]


success_rate function returns the percentage of succesfull predictions

In [5]:
def success_rate(y_test_pred, y_test, N):
    successes = 0
    for i in range(len(y_test)):
        if np.abs(y_test_pred[i] - y_test[i]) < N:
            successes = successes + 1
    succession = (successes/len(y_test))*100
    return succession

Training the model with Linear Regression

In [6]:
from sklearn import linear_model
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import cross_val_predict

# Create linear regression object
regr = linear_model.LinearRegression()
cells_y_train_pred = cross_val_predict(regr, cells_X_train_scaled, cells_y_train)

# Train the model using the training sets
regr.fit(cells_X_train_scaled, cells_y_train)
cells_y_test_pred = regr.predict(cells_X_test_scaled)

# Showing the results
print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

R^2: 0.49308236358990576
Percentage of Succession: 42.634730538922156%


Training the model without normalization to see if there is any improvement

In [7]:
#data without scaling
cells_y_train_predNotScaled = cross_val_predict(regr, cells_X_train, cells_y_train)

# Train the model using the training sets
regr.fit(cells_X_train, cells_y_train)
cells_y_test_predNotScaled = regr.predict(cells_X_test)

# Showing the results
print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_predNotScaled)))
print("Percentage of Succession: " +str(success_rate(cells_y_test_predNotScaled, cells_y_test, 1))+ "%")

R^2: 0.4930823635899051
Percentage of Succession: 42.634730538922156%


Ridge Regression with free a and coefficients. The least important variable in our model is the shucked weight (small coefficients)

In [8]:
from sklearn.model_selection import GridSearchCV
regr2  = linear_model.Ridge(alpha=10**4)
cells_y_train_pred = cross_val_predict(regr2, cells_X_train_scaled, cells_y_train, cv=10)

regr2.fit(cells_X_train_scaled, cells_y_train)
cells_y_test_pred = regr2.predict(cells_X_test_scaled)

print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

print("Coef LinReg: " +str(regr.coef_))
print("Coef Ridge: " +str(regr2.coef_))

R^2: 0.2673003418089348
Percentage of Succession: 30.898203592814372%
Coef LinReg: [[ -2.91153646  15.52247779  10.22353942   9.5295705  -20.64403838
  -10.19204681   8.85323305]]
Coef Ridge: [[0.20581873 0.22661863 0.23286621 0.18589217 0.06597421 0.15260696
  0.2945147 ]]


α=10^6 -> no improvement

In [9]:
from sklearn.model_selection import GridSearchCV
regr2  = linear_model.Ridge(alpha=10**6)
cells_y_train_pred = cross_val_predict(regr2, cells_X_train_scaled, cells_y_train, cv=10)

regr2.fit(cells_X_train_scaled, cells_y_train)
cells_y_test_pred = regr2.predict(cells_X_test_scaled)

print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

R^2: -0.011977030415008638
Percentage of Succession: 31.736526946107784%


Ridge Regression with polynomial features of second degree in the normalized data

In [10]:
from sklearn.preprocessing import PolynomialFeatures
polyModel = PolynomialFeatures(degree = 2)

cells_X_train_poly = polyModel.fit_transform(cells_X_train_scaled)
cells_X_test_poly = polyModel.fit_transform(cells_X_test_scaled)

# normalize the polynomial features
datascaler = StandardScaler().fit(cells_X_train_poly)
cells_X_train_poly = datascaler.transform(cells_X_train_poly)
cells_X_test_poly = datascaler.transform(cells_X_test_poly)

regr3  = linear_model.Ridge(alpha=10**4)
cells_y_train_pred = cross_val_predict(regr3, cells_X_train_poly, cells_y_train, cv=10)

regr3.fit(cells_X_train_poly, cells_y_train)
cells_y_test_pred = regr3.predict(cells_X_test_poly)

print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

R^2: 0.3129306870932579
Percentage of Succession: 33.17365269461078%


Trying 20 values in logarithmic scale from 10^-6 to 10^4 we see improvement in our metrics

In [11]:
params = { 
           'alpha': np.logspace(-6, 4, 20) # try 20 values in logarithmic scale from 10^-6 to 10^4
         }
print("alphas: ")
print(params['alpha'])

regr5  = linear_model.Ridge() 
grid = GridSearchCV(estimator=regr5, cv = 10, param_grid=params) 
grid.fit(cells_X_train_poly, cells_y_train) 
cells_y_train_pred = grid.predict(cells_X_train_poly)
cells_y_test_pred = grid.predict(cells_X_test_poly)

print("Best lambda: " +str(grid.best_params_['alpha']))
print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

alphas: 
[1.00000000e-06 3.35981829e-06 1.12883789e-05 3.79269019e-05
 1.27427499e-04 4.28133240e-04 1.43844989e-03 4.83293024e-03
 1.62377674e-02 5.45559478e-02 1.83298071e-01 6.15848211e-01
 2.06913808e+00 6.95192796e+00 2.33572147e+01 7.84759970e+01
 2.63665090e+02 8.85866790e+02 2.97635144e+03 1.00000000e+04]
Best lambda: 78.47599703514607
R^2: 0.554291674273474
Percentage of Succession: 42.9940119760479%


k-Nearest Neighbours with uniform και distance. Improvement on succession rate

In [12]:
from sklearn.neighbors import KNeighborsRegressor
regr4 = KNeighborsRegressor()
params = { 
           'n_neighbors': range(1,10),
            'weights': ['uniform', 'distance']
         }
grid2 = GridSearchCV(estimator=regr4, cv = 10, param_grid=params) 
grid2.fit(cells_X_train_scaled, cells_y_train) 
cells_y_test_pred = grid2.predict(cells_X_test_scaled)

print("Best Score: " +str(grid2.best_score_))
print("Best k: " +str(grid2.best_params_['n_neighbors']))
print("Best weighting strategy: " +str(grid2.best_params_['weights']))
print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

Best Score: 0.38079647749068585
Best k: 9
Best weighting strategy: distance
R^2: 0.554291674273474
Percentage of Succession: 46.82634730538922%


Ridge Regression in not normalized data - Polynomial Features of second degree

In [13]:
from sklearn.preprocessing import PolynomialFeatures
polyModel = PolynomialFeatures(degree = 2)

cells_X_train_poly = polyModel.fit_transform(cells_X_train)
cells_X_test_poly = polyModel.fit_transform(cells_X_test)

# normalize the polynomial features
datascaler = StandardScaler().fit(cells_X_train_poly)
cells_X_train_poly = datascaler.transform(cells_X_train_poly)
cells_X_test_poly = datascaler.transform(cells_X_test_poly)

regr3  = linear_model.Ridge(alpha=10**4)
cells_y_train_pred = cross_val_predict(regr3, cells_X_train_poly, cells_y_train, cv=10)

regr3.fit(cells_X_train_poly, cells_y_train)
cells_y_test_pred = regr3.predict(cells_X_test_poly)

print("R^2: " +str(explained_variance_score(cells_y_train, cells_y_train_pred)))
print("Percentage of Succession: "+str(success_rate(cells_y_test_pred, cells_y_test, 1))+ "%")

R^2: 0.32118187733465076
Percentage of Succession: 32.93413173652694%


Simple Lineal Regression had the best results. We can understand that the relationship between the characteristics is linear.